# School Locations Processing
We have 3 different files with school location information, and each file has slightly different contents. Need to compare contents and resolve what our final/true list of geo-locatable schools is.

In [1]:
import geopandas as gpd
import pandas as pd

## Load Raw Data

### School Point Locations
Data source: https://data.cityofnewyork.us/Education/School-Point-Locations/jfju-ynrr/about_data

Last updated: November 26, 2024

Annoyingly, the data dictionary on the above linked page doesn't match the data itself, so we're left to guess on the meaning of some of these fields. Also, the description on the above linked page says this data contains Address, Principal, and Principal contact info, but that isn't in here.

In [2]:
school_points_gdf = gpd.read_file('../data/raw_data/DOE/School Locations/School Point Locations/SchoolPoints_APS_2024_08_28/SchoolPoints_APS_2024_08_28.shp')
school_points_gdf.rename(columns={'Location_C': 'Location Code', 'Name': 'Location Name'}, inplace=True)
school_points_gdf

DataSourceError: ../data/raw_data/DOE/School Locations/School Point Locations/SchoolPoints_APS_2024_08_28/SchoolPoints_APS_2024_08_28.shp: No such file or directory

### LCGMS
Last updated: November 26, 2024

This data has more robust fields in it related to grades, address, open date, principal contact info, etc. But there is a discrepancy in the records included in the geocoded vs. non-geocoded files. Not sure yet if there are any other discrepancies between these two files but need to figure that out.

#### Non-geocoded LCGMS data
Source: https://www.nycenet.edu/PublicApps/LCGMS.aspx

In [ ]:
lcgms_df = pd.read_excel('../data/raw_data/DOE/School Locations/LCGMS/LCGMS_SchoolData_20250806_0112.xlsx', dtype=str, engine='openpyxl')
lcgms_df

#### Geocoded LCGMS data
Source: https://data.cityofnewyork.us/Education/NYC-DOE-Public-School-Location-Information/3bkj-34v2/about_data

In [ ]:
lcgms_geocoded_df = pd.read_csv('../data/raw_data/DOE/School Locations/LCGMS/LCGMS_SchoolData_additional_geocoded_fields_added_.csv', encoding='latin-1')
lcgms_geocoded_df
# lcgms_geocoded_gdf = gpd.GeoDataFrame(lcgms_geocoded_df, geometry=gpd.GeoSeries.from_xy(lcgms_geocoded_df['lon'], lcgms_geocoded_df['lat']), crs=4326)

## Investigate Discrepancies

In [ ]:
# TODO: need to somehow summarize for our non-technical folks how/that there are discrepances between these datasets so they can maybe run those down by emailing DOE officials or something. Wouldn't want us suggesting fake schools to the Z campaign or something weird like that.
# Maybe they can also just do a manual fact-check on the records that are different between datasets?

### School Points vs. LCGMS non-geocoded

The non-geocoded LCGMS data seems like it is the more "official" data compared to School Points and geocoded LCGMS, due to it being the spreadsheet downloaded from following links on the official LCGMS page [here](https://infohub.nyced.org/in-our-schools/operations/lcgms) rather than being sort of a sneaky extra dataset included on NYC open data (i.e. the geocoded one) or a poorly documented Shapefile on NYC Open Data. So, I think it makes sense to trust/prefer the non-geocoded LCGMS data over the geocoded data and attempt to map the non-geocoded LCGMS data somehow. The easiest way to do that would be to attach it to the school points layer, but we need to figure out how doable that is first (i.e. discrepancies in that potential join).

In [ ]:
# Show columns that are in both school_points_gdf and lcgms_df
school_points_gdf.columns.intersection(lcgms_df.columns)

In [ ]:
# Show instances where Location Name is different between school_points_gdf and lcgms_df
school_points_gdf[['Location Code', 'Location Name']].merge(
    lcgms_df[['Location Code', 'Location Name']],
    on='Location Code',
    how='inner',
    suffixes=('_school', '_lcgms')
).query('`Location Name_school` != `Location Name_lcgms`')

#### How many LCGMS records are NOT in School Points? (i.e. `set(LCGMS).difference(set(School Points))`)

In [ ]:
# Show records that are NOT in school points but ARE in LCGMS
lcgms_records_missing_from_school_points = school_points_gdf[['Location Code', 'Location Name']].merge(
    lcgms_df[['Location Code', 'Location Name']],
    on='Location Code',
    how='outer',
    suffixes=('_school_points', '_lcgms'),
    indicator=True
).query('_merge == "right_only"')
print("Total records in LCGMS that are NOT in School Points:", len(lcgms_records_missing_from_school_points))
lcgms_records_missing_from_school_points

Check if we can find the LCGMS records that are missing from School Points in geocoded LCGMS instead

In [ ]:
# Welp, only 1 of the LCGMS records that are missing from school points are actually in the geocoded LCGMS data, but the lat/lon for that record is missing in geocoded LCGMS as well. So we won't have a use for geocoded LCGMS.
# TODO: have someone on our team research these 17 schools we can't map
lcgms_records_missing_from_school_points.drop(columns=['_merge']).merge(
    lcgms_geocoded_df[['Location Code', 'Location Name', 'Latitude', 'Longitude']],
    on='Location Code',
    how='left',
    suffixes=('_missing_from_school_points', '_geocoded'),
    indicator=True
).query('_merge == "both"')


#### How many School Points records are NOT in LCGMS? (i.e. `set(School Points).difference(set(LCGMS))`)

In [ ]:
school_points_records_missing_from_lcgms = school_points_gdf[['Location Code', 'Location Name']].merge(
    lcgms_geocoded_df[['Location Code', 'Location Name']],
    on='Location Code',
    how='outer',
    suffixes=('_school_points', '_lcgms'),
    indicator=True
).query('_merge == "left_only"')
print("Total records in School Points that are NOT in LCGMS:", len(school_points_records_missing_from_lcgms))
school_points_records_missing_from_lcgms

Check if we can find the School Points records that are missing from LCGMS in geocoded LCGMS instead

In [ ]:
# phewf - no records from geocoded LCGMS that would have to be added to school points even after joining non-geocoded LCGMS onto school points.
school_points_records_missing_from_lcgms.drop(columns=['_merge']).merge(
    lcgms_geocoded_df[['Location Code', 'Location Name']],
    on='Location Code',
    how='left',
    suffixes=('_missing_from_lcgms', '_geocoded'),
    indicator=True
).query('_merge == "both"')

#### Are there any records in geocoded LCGMS that are NOT in the joined result of non-geocoded LCGMS + School Points?

Thankfully, no.

In [ ]:
# Show records that are in geocoded LCGMS but NOT in the joined result of non-geocoded LCGMS + School Points
school_points_gdf.merge(
    lcgms_df[['Location Code', 'Location Name']],
    on='Location Code',
    how='outer',
).merge(
    lcgms_geocoded_df[['Location Code', 'Location Name']],
    on='Location Code',
    how='left',
    suffixes=('', '_geocoded'),
    indicator=True
).query('_merge == "right_only"')

### LCGMS non-geocoded vs. LCGMS geocoded

The TL;DR here is that the delta between these two datasets is really befuddling, especially since it seems the geocoded one should be the exact same as the non-geocoded one but for additional lat/lon fields. I don't know the rhyme nor reason for these discrepancies, and so I think we should just prefer/trust the more official-looking one wherever possible, which is the non-geocoded LCGMS.

Would love for the civil servants who made these datasets to explain why they are different.

In [ ]:
# NOTE from LCGMS data dict: "LOCATION CODE: a unique identifier that can include schools, administrative offices, learning communities, etc.  When the Learning_Community_Name = ‘School’, the Location_Code is a combination of the borough code and the school number.""

In [ ]:
# Show fields that are in lcgms_geocoded_df but NOT in lcgms_df
lcgms_geocoded_df.columns.difference(lcgms_df.columns)

In [ ]:
# Show fields that are in lcgms_df but NOT in lcgms_geocoded_df
lcgms_df.columns.difference(lcgms_geocoded_df.columns)

In [ ]:
# Compare data in shared columns between lcgms_df and lcgms_geocoded_df
shared_columns = set(lcgms_df.columns).intersection(set(lcgms_geocoded_df.columns))
print(f"Shared columns: {sorted(shared_columns)}")

# Merge the dataframes on Location Code to compare shared columns
comparison_df = lcgms_df.merge(
    lcgms_geocoded_df, 
    on='Location Code', 
    how='inner', 
    suffixes=('_non_geocoded', '_geocoded')
)

# Check for differences in each shared column (excluding Location Code which is the join key)
shared_data_columns = [col for col in shared_columns if col != 'Location Code']
differences_summary = {}

for col in shared_data_columns:
    col_non_geo = f"{col}_non_geocoded"
    col_geo = f"{col}_geocoded"
    
    # Count records where values differ (handling NaN values)
    different_mask = (
        (comparison_df[col_non_geo].fillna('') != comparison_df[col_geo].fillna('')) |
        (comparison_df[col_non_geo].isna() != comparison_df[col_geo].isna())
    )
    
    num_differences = different_mask.sum()
    differences_summary[col] = num_differences
    
    if num_differences > 0:
        print(f"\n{col}: {num_differences} differences found")
        # Show first few examples of differences
        diff_examples = comparison_df[different_mask][['Location Code', col_non_geo, col_geo]].head()
        print(diff_examples)

print(f"\nSummary of differences:")
for col, count in differences_summary.items():
    print(f"{col}: {count} differences")

In [ ]:
# Show records where Location Code matches but Location Name does not match
lcgms_df[['Location Code', 'Location Name']].merge(
    lcgms_geocoded_df[['Location Code', 'Location Name']], 
    on='Location Code', 
    how='inner', 
    suffixes=('_non-geocoded', '_geocoded')
).query('`Location Name_non-geocoded` != `Location Name_geocoded`')

## Join Data

### Outer Join LCGMS with School Points

In [ ]:
# For our final school points data (at least for now), join LCGMS onto school points
school_points_with_lcgms = school_points_gdf.merge(
    # Just going to use Location Name from School Points bc above check showed that there's minimal differences there.
    lcgms_df.drop(columns=['Location Name']),
    on='Location Code',
    how='outer',
    indicator=True
)

# Keep a column that indicates whether the record is missing from LCGMS or not so we can select just LCGMS records if we find out that the school points data is outdated or inaccurate compared to LCGMS.
school_points_with_lcgms.rename(columns={'_merge': 'in_LCGMS'}, inplace=True)
school_points_with_lcgms['in_LCGMS'] = school_points_with_lcgms['in_LCGMS'].str.contains('both|right_only')

school_points_with_lcgms

# Clean Data

In [ ]:
pd.set_option('display.max_columns', None)
school_points_with_lcgms

## Cleaning up specific fields

In [ ]:
# TODO: not sure why we have disagreements between Building Code and Building_C
school_points_with_lcgms[
    (school_points_with_lcgms['Building_C'] != school_points_with_lcgms['Building Code'])
    # & school_points_with_lcgms['Building Code'].notna()
    # & school_points_with_lcgms['Building_C'].notna()
][['Location Code', 'Location Name', 'Building Code', 'Building_C', 'in_LCGMS']]

In [ ]:
# For now, we're going to keep the Building Code from LCGMS unless NaN
school_points_with_lcgms['Building Code'] = school_points_with_lcgms['Building Code'
                                                                     ].fillna(school_points_with_lcgms['Building_C'])

In [ ]:
# TODO: need to go through all the LCGMS columns and figure out which ones we can drop
# Drop unnecessary columns
cols_to_drop = [
    'Geographic', # This is from School Points and I have no idea what it means.
    'Building_C',  # This is the Building Code from School Point, which is duplicate
]
school_points_with_lcgms.drop(columns=cols_to_drop, inplace=True)

# Deal with Duplicate geometries

Basically half of the points are from duplicate locations. Wondering if we can bring in the buildings data to get more precise locations and fix this. Otherwise, we don't end up seeing half of the points on the map because they're on top of each other.

In [ ]:
# TODO: is this still true if we use the lat/on instead of native shapefile geometry?

In [ ]:
# Ok at least with these 3 examples, they all appear to just be different schools that 
# share an address:
#    - CSI High School for International Studies
#    - Gaynor McCown Expeditionary Learning School
#    - Marsh Avenue School for Expeditionary Learning
school_points_with_lcgms[
    (school_points_with_lcgms.duplicated(subset=['geometry'], keep=False))
    & school_points_with_lcgms['geometry'].notna()
    ].sort_values(by='geometry')

## Load OTI Buildings Data

In [ ]:
oti_buildings_gdf = gpd.read_file('../data/raw_data/OTI/BUILDING_20250911.geojson')

In [ ]:
school_points_with_lcgms.columns

In [ ]:
# TODO: what is the difference between base_bbl and mappluto_bbl?
oti_buildings_gdf.columns

In [ ]:
school_points_with_lcgms.head()

In [ ]:
oti_buildings_gdf['base_bbl'].head()

In [ ]:
school_points_with_lcgms.loc[0, 'Borough Block Lot']

In [ ]:
oti_buildings_gdf[oti_buildings_gdf['base_bbl'] == '3007550022'].explore()#.explore(tiles='CartoDB positron',
                # popup=['Location Name', 'Community District', 'Council District',
                #        'Principal Name', 'Principal Title', 'Principal Phone Number'],
                # tooltip=['Location Name'],  # Show on hover
                # legend=True,
                # style_kwds={'fillOpacity': 0.7, 'weight': 1}
#)

In [ ]:
# 94% of school points have at least one match in OTI buildings based on BBL. The problem is many have multiple matches.
print('pct of school points with matching BBL in OTI buildings:', school_points_with_lcgms['Borough Block Lot'].isin(oti_buildings_gdf['base_bbl']).sum() / len(school_points_with_lcgms))

In [ ]:
school_points_with_lcgms['Location Code'].nunique()

In [ ]:
# show number of unique geometries in school_points_with_lcgms
school_points_with_lcgms['geometry'].nunique()

In [ ]:
# show number of unique geometries in OTI buildings that match to school points based on BBL, when dropping dupes by keeping largest matched building
# hmmm ok so this is actually worse than LCGMS alone...1320 unique geometries here, but 1381 before with just LCGMS
school_points_with_lcgms.merge(
    oti_buildings_gdf, 
    left_on='Borough Block Lot', 
    right_on='base_bbl', 
    how='left', 
    suffixes=['_lcgms', '_oti']
).sort_values(
    by='shape_area',
    # keep the largest building match
    ascending=False
    ).drop_duplicates(
        subset=['Location Code'],
        keep='first')['geometry_oti'].nunique()

In [ ]:
# I want to drop dupes on Location Code, choosing a unique geometry where possible.
# So if a duped Location Code has a geometry that isn't used by any other Location Code, prefer that one.
# If none of the dupes have a unique geometry, just keep the first one.
school_points_with_lcgms.merge(
    oti_buildings_gdf,
    left_on='Borough Block Lot',
    right_on='base_bbl',
    how='left',
    suffixes=['_lcgms', '_oti']
).drop_duplicates(
        subset=['Location Code'],
        keep='first')['geometry_oti'].nunique()

In [ ]:
# Need to figure out what I can drop from OTI. Then figure out how I join to school points.

# Sanity Check: Visualize Data with Geopandas

In [ ]:
school_points_with_lcgms.explore(tiles='CartoDB positron',
                popup=['Location Name', 'Community District', 'Council District',
                       'Principal Name', 'Principal Title', 'Principal Phone Number'],
                tooltip=['Location Name'],  # Show on hover
                legend=True,
                style_kwds={'fillOpacity': 0.7, 'weight': 1}
)

# Export to `processed_data` folder as GeoJSON

In [ ]:
school_points_with_lcgms.to_file(
    '../data/processed_data/school_points_with_lcgms.geojson',
      driver='GeoJSON'
      )